<a href="https://colab.research.google.com/github/siavashadpey/gpu_intro/blob/master/cs179_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 2 of CS179 (http://courses.cms.caltech.edu/cs179/)

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gl6srsfo
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gl6srsfo
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=8936d1d622f03daa9664a873352a6bd4f15b7149dccc180bbd7cc37d710703f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-rgw5azou/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [32]:
%%cu

#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <cstring>
#include <string>
#include <unistd.h> // sleep, fork, getpid
#include <signal.h> // kill
#include <cstdio> // printf
#include <stdlib.h> // popen, pclose, atoi, fread

#include <cuda_runtime.h>

enum TransposeImplementation { NAIVE, SHMEM, OPTIMAL };

/*
 =============================== STR: DEVICE    ================================
*/
/*
 * TODO for all kernels (including naive):
 * Leave a comment above all non-coalesced memory accesses and bank conflicts.
 * Make it clear if the suboptimal access is a read or write. If an access is
 * non-coalesced, specify how many cache lines it touches, and if an access
 * causes bank conflicts, say if its a 2-way bank conflict, 4-way bank
 * conflict, etc.
 *
 * Comment all of your kernels.
 */

/*
 * Each block of the naive transpose handles a 64x64 block of the input matrix,
 * with each thread of the block handling a 1x4 section and each warp handling
 * a 32x4 section.
 *
 * If we split the 64x64 matrix into 32 blocks of shape (32, 4), then we have
 * a block matrix of shape (2 blocks, 16 blocks).
 * Warp 0 handles block (0, 0), warp 1 handles (1, 0), warp 2 handles (0, 1),
 * warp n handles block (n % 2, n / 2).
 *
 * This kernel is launched with block shape (64, 16) and grid shape
 * (n / 64, n / 64) where n is the size of the square matrix.
 *
 * You may notice that we suggested in lecture that threads should be able to
 * handle an arbitrary number of elements and that this kernel handles exactly
 * 4 elements per thread. This is OK here because to overwhelm this kernel
 * it would take a 4194304 x 4194304 matrix, which would take ~17.6TB of
 * memory (well beyond what I expect GPUs to have in the next few years).
 */
__global__
void naiveTransposeKernel(const float *input, float *output, int n) {
    // DONE: do not modify code, just comment on suboptimal accesses

    const int i = threadIdx.x + 64 * blockIdx.x;
    int j = 4 * threadIdx.y + 64 * blockIdx.y;
    const int end_j = j + 4;

    // COMMENT: we can unroll j to allow for ILP
    // COMMENT: memory access to ouput is not coallesced (stride = n, not 1).
    //          it touches 32 cache lines (assuming n > 32).
    for (; j < end_j; j++)
        output[j + n * i] = input[i + n * j];
}

__global__
void shmemTransposeKernel(const float *input, float *output, int n) {
    // DONE: Modify transpose kernel to use shared memory. All global memory
    // reads and writes should be coalesced. Minimize the number of shared
    // memory bank conflicts (0 bank conflicts should be possible using
    // padding). Again, comment on all sub-optimal accesses.

    // each block takes care of a chunk of size 64 x 64
    __shared__ float data[64][65]; // 64 x 64 block with an extra column of padding

    // blockDim.x = 4*blockDim.y = 64
    int i = threadIdx.x + 64 * blockIdx.x;
    int j = 4 * threadIdx.y + 64 * blockIdx.y;
    int end_j = j + 4;
    
    // copy input to shared memory
    const int li = threadIdx.x;
    int lj = 4*threadIdx.y;
    for (; j < end_j; j++, lj++) {
        // we're using a stride of 1 for global memory --> memory coallescing
        data[lj][li] = input[i + j * n]; 
        // note: even without the padding, no bank conflicts would occur here
        // elements in data[lj][:] belong to different banks
    }

    __syncthreads(); // sync with all other threads in current block

    // reset some indices
    i = threadIdx.x + 64 * blockIdx.y;
    j = 4 * threadIdx.y + 64 * blockIdx.x;
    lj = 4*threadIdx.y;
    end_j = j + 4;
    // copy output to global memory
    for (; j < end_j; j++, lj++) {
        // we're using a stride of 1 for global memory --> memory coallescing
        output[i + j * n] = data[li][lj];
        // the padding is necesseray to prevent bank conflicts here
        // each element in data[:][lj] belongs to different banks
    }
    // COMMENT: for loops should be unrolled for ILP
}

__global__
void optimalTransposeKernel(const float *input, float *output, int n) {
    // TODO: This should be based off of your shmemTransposeKernel.
    // Use any optimization tricks discussed so far to improve performance.
    // Consider ILP and loop unrolling.

__shared__ float data[64][65]; // 64 x 64 block with an extra column of padding

    // blockDim.x = 4*blockDim.y = 64
    int i = threadIdx.x + 64 * blockIdx.x;
    int j = 4 * threadIdx.y + 64 * blockIdx.y;
    
    // copy input to shared memory
    const int li = threadIdx.x;
    const int lj = 4*threadIdx.y;
    // we've manually unrolled the for loop
    data[lj    ][li] = input[i +  j      * n]; 
    data[lj + 1][li] = input[i + (j + 1) * n]; 
    data[lj + 2][li] = input[i + (j + 2) * n]; 
    data[lj + 3][li] = input[i + (j + 3) * n]; 

    __syncthreads(); // sync with all other threads in current block

    // reset some indices
    i = threadIdx.x + 64 * blockIdx.y;
    j = 4 * threadIdx.y + 64 * blockIdx.x;

    // copy output to global memory
    // we've manually unrolled the for loop
    output[i + j       * n] = data[li][lj    ];
    output[i + (j + 1) * n] = data[li][lj + 1];
    output[i + (j + 2) * n] = data[li][lj + 2];
    output[i + (j + 3) * n] = data[li][lj + 3];
}

void cudaTranspose(
    const float *d_input,
    float *d_output,
    int n,
    TransposeImplementation type)
{
    if (type == NAIVE) {
        dim3 blockSize(64, 16);
        dim3 gridSize(n / 64, n / 64);
        naiveTransposeKernel<<<gridSize, blockSize>>>(d_input, d_output, n);
    }
    else if (type == SHMEM) {
        dim3 blockSize(64, 16);
        dim3 gridSize(n / 64, n / 64);
        shmemTransposeKernel<<<gridSize, blockSize>>>(d_input, d_output, n);
    }
    else if (type == OPTIMAL) {
        dim3 blockSize(64, 16);
        dim3 gridSize(n / 64, n / 64);
        optimalTransposeKernel<<<gridSize, blockSize>>>(d_input, d_output, n);
    }
    // Unknown type
    else
        assert(false);
}

/*
 =============================== END: DEVICE    ================================
*/

/*
 =============================== STR: UTILITIES ================================
*/

namespace TA_Utilities
{
  /* Select the least utilized GPU on this system. Estimate
     GPU utilization using GPU temperature. UNIX only. */
  void select_coldest_GPU() 
  {
      // Get the number of GPUs on this machine
      int num_devices;
      cudaGetDeviceCount(&num_devices);
      if(num_devices == 0) {
          printf("select_coldest_GPU: Error - No GPU detected\n");
          return;
      }
      // Read GPU info into buffer "output"
      const unsigned int MAX_BYTES = 10000;
      char output[MAX_BYTES];
      FILE *fp = popen("nvidia-smi &> /dev/null", "r");
      size_t bytes_read = fread(output, sizeof(char), MAX_BYTES, fp);
      pclose(fp);
      if(bytes_read == 0) {
          printf("Error - No Temperature could be read\n");
          return;
	  }
      // array to hold GPU temperatures
      int * temperatures = new int[num_devices];
      // parse output for temperatures using knowledge of "nvidia-smi" output format
      int i = 0;
      unsigned int num_temps_parsed = 0;
      while(output[i] != '\0') {
          if(output[i] == '%') {
              unsigned int temp_begin = i + 1;
              while(output[i] != 'C') {
                  ++i;
              }
              unsigned int temp_end = i;
              char this_temperature[32];
              // Read in the characters cooresponding to this temperature
              for(unsigned int j = 0; j < temp_end - temp_begin; ++j) {
                  this_temperature[j] = output[temp_begin + j];
              }
              this_temperature[temp_end - temp_begin + 1] = '\0';
              // Convert string representation to int
              temperatures[num_temps_parsed] = atoi(this_temperature);
              num_temps_parsed++;
          }
          ++i;
      }
      // Get GPU with lowest temperature
      int min_temp = 1e7, index_of_min = -1;
      for (int i = 0; i < num_devices; i++) 
      {
          int candidate_min = temperatures[i];
          if(candidate_min < min_temp) 
          {
              min_temp = candidate_min;
              index_of_min = i;
          }
      }
      // Tell CUDA to use the GPU with the lowest temeprature
      printf("Index of the GPU with the lowest temperature: %d (%d C)\n", 
          index_of_min, min_temp);
      cudaSetDevice(index_of_min);
      // Free memory and return
      delete(temperatures);
      return;
  } // end "void select_coldest_GPU()""

  /* Create a child thread that will kill the parent thread after the
     specified time limit has been exceeded */
  void enforce_time_limit(int time_limit) {
      printf("Time limit for this program set to %d seconds\n", time_limit);
      int parent_id = getpid();
      pid_t child_id = fork();
      // The fork call creates a lignering child thread that will 
      // kill the parent thread after the time limit has exceeded
      // If it hasn't already terminated.
      if(child_id == 0) // "I am the child thread"
      {
          sleep(time_limit);
          if( kill(parent_id, SIGTERM) == 0) {
              printf("enforce_time_limit.c: Program terminated"
               " for taking longer than %d seconds\n", time_limit);
          }
          // Ensure that parent was actually terminated
          sleep(2);
          if( kill(parent_id, SIGKILL) == 0) {
              printf("enforce_time_limit.c: Program terminated"
               " for taking longer than %d seconds\n", time_limit);
          } 
          // Child thread has done its job. Terminate now.
          exit(0);
      }
      else // "I am the parent thread"
      {
          // Allow the parent thread to continue doing what it was doing
          return;
      }
  } // end "void enforce_time_limit(int time_limit)


} // end "namespace TA_Utilities"

/*
 =============================== END: UTILITIES ================================
*/

/*
 =============================== STR: HOST      ================================
*/
/*
 * NOTE: You can use this macro to easily check cuda error codes
 * and get more information.
 * 
 * Modified from:
 * http://stackoverflow.com/questions/14038589/
 *         what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrChk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line,
    bool abort = true)
{
    if (code != cudaSuccess) {
        fprintf(stderr,"GPUassert: %s %s %d\n",
            cudaGetErrorString(code), file, line);
        exit(code);
    }
}

/* a and b point to n x n matrices. This method checks that A = B^T. */
void checkTransposed(const float *a, const float *b, int n) {
    bool correct = true;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            if (a[i + n * j] != b[j + n * i]) {
                correct = false;
                fprintf(stderr,
                    "Transpose failed: a[%d, %d] != b[%d, %d], %f != %f\n",
                    i, j, j, i, a[i + n * j], b[j + n * i]);
                assert(correct);
            }
        }
    }

    assert(correct);
}

/* Naive CPU transpose, takes an n x n matrix in input and writes to output. */
void cpuTranspose(const float *input, float *output, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            output[j + n * i] = input[i + n * j];
        }
    }
}

/*
 * Fills fill with random numbers is [0, 1]. Size is number of elements to
 * assign.
 */
void randomFill(float *fill, int size) {
    for (int i = 0; i < size; i++) {
        float r = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
        fill[i] = r;
    }
}

int main(int argc, char *argv[]) {

    // These functions allow you to select the least utilized GPU 
    // on your system as well as enforce a time limit on program execution.
    // Please leave these enabled as a courtesy to your fellow classmates
    // if you are using a shared computer. You may ignore or remove these 
    // functions if you are running on your local machine.
    TA_Utilities::select_coldest_GPU();
    int max_time_allowed_in_seconds = 10;
    TA_Utilities::enforce_time_limit(max_time_allowed_in_seconds);
    
    // Seed random number generator
    srand(2016);

    std::string kernel = "all";
    int size_to_run = -1;

    // Check arguments
    assert(argc <= 3);
    if (argc >= 2)
        size_to_run = atoi(argv[1]);
    if (argc == 3)
        kernel = argv[2];

    if (!(size_to_run == -1  ||
         size_to_run == 512  ||
         size_to_run == 1024 ||
         size_to_run == 2048 ||
         size_to_run == 4096))
    {
        fprintf(stderr,
            "Program only designed to run sizes 512, 1024, 2048, 4096\n");
    }

    assert(kernel == "all"     ||
        kernel == "cpu"        ||
        kernel == "gpu_memcpy" ||
        kernel == "naive"      ||
        kernel == "shmem"      ||
        kernel == "optimal");

    // Run the transpose implementations for all desired sizes (2^9 = 512, 
    // 2^12 = 4096)
    for (int _i = 9; _i < 13; _i++) {
        int n = 1 << _i;
        if (size_to_run != -1 && size_to_run != n)
            continue;

        assert(n % 64 == 0);

        cudaEvent_t start;
        cudaEvent_t stop;

#define START_TIMER() {                                                        \
            gpuErrChk(cudaEventCreate(&start));                                \
            gpuErrChk(cudaEventCreate(&stop));                                 \
            gpuErrChk(cudaEventRecord(start));                                 \
        }

#define STOP_RECORD_TIMER(name) {                                              \
            gpuErrChk(cudaEventRecord(stop));                                  \
            gpuErrChk(cudaEventSynchronize(stop));                             \
            gpuErrChk(cudaEventElapsedTime(&name, start, stop));               \
            gpuErrChk(cudaEventDestroy(start));                                \
            gpuErrChk(cudaEventDestroy(stop));                                 \
        }

        // Initialize timers
        float cpu_ms = -1;
        float gpu_memcpy = -1;
        float naive_gpu_ms = -1;
        float shmem_gpu_ms = -1;
        float optimal_gpu_ms = -1;

        // Allocate host memory
        float *input = new float[n * n];
        float *output = new float[n * n];

        // Allocate device memory
        float *d_input;
        float *d_output;
        gpuErrChk(cudaMalloc(&d_input, n * n * sizeof(float)));
        gpuErrChk(cudaMalloc(&d_output, n * n * sizeof(float)));

        // Initialize input data to random numbers in [0, 1]
        randomFill(input, n * n);

        // Copy input to GPU
        gpuErrChk(cudaMemcpy(d_input, input, n * n * sizeof(float), 
            cudaMemcpyHostToDevice));


        // CPU implementation
        if (kernel == "cpu" || kernel == "all") {
            START_TIMER();
            cpuTranspose(input, output, n);
            STOP_RECORD_TIMER(cpu_ms);

            checkTransposed(input, output, n);
            memset(output, 0, n * n * sizeof(float));

            printf("Size %d naive CPU: %f ms\n", n, cpu_ms);
        }

        // GPU memcpy (useful for comparison purposes)
        if (kernel == "gpu_memcpy" || kernel == "all") {
            START_TIMER();
            gpuErrChk(cudaMemcpy(d_output, d_input, n * n * sizeof(float),
                cudaMemcpyDeviceToDevice));
            STOP_RECORD_TIMER(gpu_memcpy);

            gpuErrChk(cudaMemset(d_output, 0, n * n * sizeof(float)));
            printf("Size %d GPU memcpy: %f ms\n", n, gpu_memcpy);
        }

        // Naive GPU implementation
        if (kernel == "naive" || kernel == "all") {
            START_TIMER();
            cudaTranspose(d_input, d_output, n, NAIVE);
            STOP_RECORD_TIMER(naive_gpu_ms);

            gpuErrChk(cudaMemcpy(output, d_output, n * n * sizeof(float), 
                cudaMemcpyDeviceToHost));
            checkTransposed(input, output, n);

            memset(output, 0, n * n * sizeof(float));
            gpuErrChk(cudaMemset(d_output, 0, n * n * sizeof(float)));

            printf("Size %d naive GPU: %f ms\n", n, naive_gpu_ms);
        }

        // shmem GPU implementation
        if (kernel == "shmem" || kernel == "all") {
            START_TIMER();
            cudaTranspose(d_input, d_output, n, SHMEM);
            STOP_RECORD_TIMER(shmem_gpu_ms);

            gpuErrChk(cudaMemcpy(output, d_output, n * n * sizeof(float), 
                cudaMemcpyDeviceToHost));
            checkTransposed(input, output, n);

            memset(output, 0, n * n * sizeof(float));
            gpuErrChk(cudaMemset(d_output, 0, n * n * sizeof(float)));

            printf("Size %d shmem GPU: %f ms\n", n, shmem_gpu_ms);
        }

        // Optimal GPU implementation
        if (kernel == "optimal"    || kernel == "all") {
            START_TIMER();
            cudaTranspose(d_input, d_output, n, OPTIMAL);
            STOP_RECORD_TIMER(optimal_gpu_ms);

            gpuErrChk(cudaMemcpy(output, d_output, n * n * sizeof(float), 
                cudaMemcpyDeviceToHost));
            checkTransposed(input, output, n);

            memset(output, 0, n * n * sizeof(float));
            gpuErrChk(cudaMemset(d_output, 0, n * n * sizeof(float)));

            printf("Size %d optimal GPU: %f ms\n", n, optimal_gpu_ms);
        }

        // Free host memory
        delete[] input;
        delete[] output;

        // Free device memory
        gpuErrChk(cudaFree(d_input));
        gpuErrChk(cudaFree(d_output));

        printf("\n");
    }
}

/*
 =============================== END: HOST      ================================
*/


Index of the GPU with the lowest temperature: 0 (0 C)
Time limit for this program set to 10 seconds
Size 512 naive CPU: 1.901312 ms
Size 512 GPU memcpy: 0.042560 ms
Size 512 naive GPU: 0.135296 ms
Size 512 shmem GPU: 0.032160 ms
Size 512 optimal GPU: 0.026112 ms

Size 1024 naive CPU: 7.437024 ms
Size 1024 GPU memcpy: 0.090400 ms
Size 1024 naive GPU: 0.195040 ms
Size 1024 shmem GPU: 0.088064 ms
Size 1024 optimal GPU: 0.070944 ms

Size 2048 naive CPU: 38.028671 ms
Size 2048 GPU memcpy: 0.297824 ms
Size 2048 naive GPU: 0.735008 ms
Size 2048 shmem GPU: 0.281696 ms
Size 2048 optimal GPU: 0.259936 ms

Size 4096 naive CPU: 239.744156 ms
Size 4096 GPU memcpy: 1.064576 ms
Size 4096 naive GPU: 2.878432 ms
Size 4096 shmem GPU: 1.092800 ms
Size 4096 optimal GPU: 1.018912 ms

Index of the GPU with the lowest temperature: 0 (0 C)
Time limit for this program set to 10 seconds
enforce_time_limit.c: Program terminated for taking longer than 10 seconds
enforce_time_limit.c: Program terminated for taking